# TFM robusto: seleção de variáveis na interseção de LASSO e p-valor

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import statsmodels.api as sm

# Dados

In [2]:
# Lendo meta_data

meta_data = pd.read_csv('../../../data/meta-data-full.csv')
meta_data.drop(['tcode', 'transf_code'], axis = 1, inplace = True)
meta_data

,group,category,code,name_df,name,source,frequency,original,order,transf,days_lag,where
0,0,Produção,22099,gdp,PIB trimestral - Dados observados - preços de ...,IBGE,Q,Index,NaN,pctchg_12,63.0,BCB
1,0,Produção,24363,ibc_br,Índice de Atividade Econômica do Banco Central...,BCB,M,Index,NaN,diff_12,52.0,BCB
2,0,Produção,4380,mon_gdp,PIB mensal - Valores correntes (R$ milhões)\t,BCB,M,R$,NaN,diff_12,48.0,BCB
3,0,Produção,21859,ind_prod,Indicadores da produção (2012=100) - Produção ...,IBGE,M,Index,NaN,diff_12,30.0,BCB
4,0,Produção,21861,ind_prod_ext,Indicadores da produção (2012=100) - Produção ...,IBGE,M,Index,NaN,diff_12,30.0,BCB
...,...,...,...,...,...,...,...,...,...,...,...,...
190,9,Google Trends,empreendedor,gt_empreendedor,Google Trends - Empreendedor,Google,M,Trend Index,NaN,diff_diff_12,NaN,gtrendsR
191,9,Google Trends,abrir empresa,gt_empresa,Google Trends - Abrir empresa,Google,M,Trend Index,NaN,diff_12,NaN,gtrendsR
192,9,Google Trends,credito,gt_credito,Google Trends - Crédito,Google,M,Trend Index,NaN,diff_diff_12,NaN,gtrendsR
193,9,Google Trends,dolar,gt_dolar,Google Trends - Dólar,Google,M,Trend Index,NaN,diff_12,NaN,gtrendsR


In [3]:
# Carregando dados tratados - CONJUNTO COMPLETO

data = pd.read_csv('../../../data/data_full_tf12.csv')
# data['Date'] = pd.date_range(start = '2002-03-01', end = '2022-09-01', freq = 'QS-DEC') --> se fosse para trimestre 
data['Date'] = pd.date_range(start = '2002-01-01', end = '2022-09-01', freq = 'MS')
data.set_index('Date', inplace = True)

'''# Agregando as séries pela média e substituindo NAs pela mediana

qdata = (data.groupby(pd.PeriodIndex(data.index, freq = 'Q'), axis = 0)).mean()
cols = qdata.columns'''

data

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,...,gt_politica,gt_aluguel,gt_seguro,gt_governo,gt_viagem,gt_empreendedor,gt_empresa,gt_credito,gt_dolar,gt_financiamento
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,0.375,0.750,0.375,0.000,1.375,-0.250,-1.125,0.000,2.375,1.250
2002-02-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,0.375,0.750,0.375,0.000,1.375,-0.250,-1.125,0.000,2.375,1.250
2002-03-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,0.375,0.750,0.375,0.000,1.375,-0.250,-1.125,0.000,2.375,1.250
2002-04-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,0.375,0.750,0.375,0.000,1.375,-0.250,-1.125,0.000,2.375,1.250
2002-05-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,0.375,0.750,0.375,0.000,1.375,-0.250,-1.125,0.000,2.375,1.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,NaN,5.07,101660.0,-0.3,-10.4,1.3,6.4,-2.3,2.2,-3.4,...,-1.000,10.625,-4.125,4.875,11.125,-6.625,-2.125,-0.625,26.625,-3.875
2022-06-01,0.036869,4.44,102487.4,-0.6,-5.9,0.2,-0.4,-3.0,3.8,2.0,...,-1.000,7.500,4.000,2.750,8.500,2.000,2.375,-8.875,-2.375,-9.750
2022-07-01,NaN,6.44,99509.7,-0.4,-4.3,0.2,-6.6,-1.8,3.7,-0.8,...,1.125,7.625,-4.125,0.000,12.500,2.250,-1.750,-1.375,7.875,-7.375


In [4]:
# CONJUNTO DE DADOS DE P-VALOR

pvalor = pd.read_csv('../../../data/data_target_full_tf12.csv')
# data['Date'] = pd.date_range(start = '2002-03-01', end = '2022-09-01', freq = 'QS-DEC') --> se fosse para trimestre 
pvalor['Date'] = pd.date_range(start = '2002-01-01', end = '2022-09-01', freq = 'MS')
pvalor.set_index('Date', inplace = True)
pvalor

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,...,gt_sports,gt_crise,gt_desemprego,gt_segdesemp,gt_economia,gt_emprestimo,gt_aluguel,gt_viagem,gt_dolar,gt_financiamento
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-02-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-03-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-04-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-05-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,NaN,5.07,101660.0,-0.3,-10.4,1.3,6.4,-2.3,2.2,-3.4,...,7.250,3.125,-1.875,-2.125,-0.375,-6.000,10.625,11.125,26.625,-3.875
2022-06-01,0.036869,4.44,102487.4,-0.6,-5.9,0.2,-0.4,-3.0,3.8,2.0,...,9.625,1.000,2.000,0.375,-1.875,-0.250,7.500,8.500,-2.375,-9.750
2022-07-01,NaN,6.44,99509.7,-0.4,-4.3,0.2,-6.6,-1.8,3.7,-0.8,...,6.500,1.875,2.125,1.625,-0.625,14.250,7.625,12.500,7.875,-7.375


In [5]:
# CONJUNTO DE DADOS DE LASSO

lasso = pd.read_csv('../../../data/data_target_lasso_full_tf12.csv')
# data['Date'] = pd.date_range(start = '2002-03-01', end = '2022-09-01', freq = 'QS-DEC') --> se fosse para trimestre 
lasso['Date'] = pd.date_range(start = '2002-01-01', end = '2022-09-01', freq = 'MS')
lasso.set_index('Date', inplace = True)
lasso

,gdp,ibc_br,ind_prod,hrs_wkd_ind_transf,nondur_congoods_sales,vehi_total_sales_fenabrave,pass_vehi_sales,energy_total,ind_real_sales,net_debt_per_gdp_states,...,ind_prod_nondur_goods,gt_finance,gt_empresa,total_sales,gt_oncomm,wage_mass_ind,gt_empreendedor,imp_comgood_fob,brent_oil_price,road_flow_gross_vehi
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,2.45,0.7,0.0,2.51,6619.0,3993.0,976.0,0.7,0.005,...,0.5,1.75,-1.125,3.33,0.125,0.0,-0.250,-0.359005,0.013375,-0.04
2002-02-01,NaN,2.45,0.7,0.0,2.51,6619.0,3993.0,976.0,0.7,0.005,...,0.5,1.75,-1.125,3.33,0.125,0.0,-0.250,-0.359005,0.013375,-0.04
2002-03-01,NaN,2.45,0.7,0.0,2.51,6619.0,3993.0,976.0,0.7,0.005,...,0.5,1.75,-1.125,3.33,0.125,0.0,-0.250,-0.359005,0.013375,-0.04
2002-04-01,NaN,2.45,0.7,0.0,2.51,6619.0,3993.0,976.0,0.7,0.005,...,0.5,1.75,-1.125,3.33,0.125,0.0,-0.250,-0.359005,0.013375,-0.04
2002-05-01,NaN,2.45,0.7,0.0,2.51,6619.0,3993.0,976.0,0.7,0.005,...,0.5,1.75,-1.125,3.33,0.125,0.0,-0.250,-0.359005,0.013375,-0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,NaN,5.07,-0.3,3.1,-0.45,-1550.0,-3180.0,1229.0,0.7,0.120,...,3.4,0.50,-2.125,-0.16,1.375,1.5,-6.625,-90.055103,0.013375,-1.99
2022-06-01,0.036869,4.44,-0.6,3.4,2.11,-4361.0,288.0,318.0,1.2,0.430,...,4.0,-9.75,2.375,-0.13,0.750,1.7,2.000,93.031865,0.050027,-2.64
2022-07-01,NaN,6.44,-0.4,1.8,-0.27,6549.0,11945.0,1008.0,-1.7,0.060,...,4.0,-4.00,-1.750,-5.58,0.625,6.4,2.250,-111.393375,-0.041808,1.44


# Selecionando pela interseção

In [6]:
cols_lasso = list(lasso.columns)
cols_pvalue = list(pvalor.columns)
robust_selection = list(set(cols_pvalue) & set(cols_lasso))

In [7]:
data = data[robust_selection]
data.to_csv('data_full_robust_tf12.csv')
data

,vehi_total_sales_fenabrave,imp_comgood_fob,energy_total,road_flow_gross_vehi,gt_viagem,total_sales,ibc_br,caged_ind_ext,ind_real_sales,gdp,ind_prod,ipca_ex,offcomm_sales,hrs_wkd_ind_transf,pass_vehi_sales,caged_agro,books_sales,wage_mass_ind,expec_exr_usd,ind_prod_nondur_goods
Date,,,,,,,,,,,,,,,,,,,,
2002-01-01,6619.0,-0.359005,976.0,-0.04,1.375,3.33,2.45,-71.0,0.7,NaN,0.7,-0.01,5.55,0.0,3993.0,20926.0,0.31,0.0,0.0500,0.5
2002-02-01,6619.0,-0.359005,976.0,-0.04,1.375,3.33,2.45,-71.0,0.7,NaN,0.7,-0.01,5.55,0.0,3993.0,20926.0,0.31,0.0,0.0500,0.5
2002-03-01,6619.0,-0.359005,976.0,-0.04,1.375,3.33,2.45,-150.0,0.7,NaN,0.7,-0.01,5.55,0.0,3993.0,20926.0,0.31,0.0,0.0500,0.5
2002-04-01,6619.0,-0.359005,976.0,-0.04,1.375,3.33,2.45,583.0,0.7,NaN,0.7,-0.01,5.55,0.0,3993.0,20926.0,0.31,0.0,0.0500,0.5
2002-05-01,6619.0,-0.359005,976.0,-0.04,1.375,3.33,2.45,68.0,0.7,NaN,0.7,-0.01,5.55,0.0,3993.0,20926.0,0.31,0.0,0.0500,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,-1550.0,-90.055103,1229.0,-1.99,11.125,-0.16,5.07,-15.0,0.7,NaN,-0.3,0.53,2.10,3.1,-3180.0,85453.0,17.80,1.5,-0.2544,3.4
2022-06-01,-4361.0,93.031865,318.0,-2.64,8.500,-0.13,4.44,-157.0,1.2,0.036869,-0.6,0.32,1.85,3.4,288.0,82143.0,2.19,1.7,0.0100,4.0
2022-07-01,6549.0,-111.393375,1008.0,1.44,12.500,-5.58,6.44,212.0,-1.7,NaN,-0.4,-0.25,-0.23,1.8,11945.0,72719.0,8.83,6.4,0.1000,4.0


# Verificando os grupos de variáveis selecionadas

In [8]:
series = list(data.columns)
n_series = np.arange(0, (data.shape[1]))
series_df = pd.DataFrame(n_series, index = series)

In [9]:
# Criando df para mapear grupo no dataframe de contribuição dos componentes

map_comp = meta_data[['category', 'name_df']].copy()
map_comp.set_index('name_df', inplace = True)
map_comp = map_comp.iloc[1:]

robust_data = pd.merge(series_df, map_comp, left_index=True, right_index=True)
robust_data['category'].value_counts()

Vendas                 5
Produção               5
Mercado de Trabalho    4
Setor Externo          1
Energia                1
Google Trends          1
Preços                 1
Expectativas           1
Name: category, dtype: int64

In [10]:
print(robust_data['category'].value_counts().to_latex())

\begin{tabular}{lr}
\toprule
{} &  category \\
\midrule
Vendas              &         5 \\
Produção            &         5 \\
Mercado de Trabalho &         4 \\
Setor Externo       &         1 \\
Energia             &         1 \\
Google Trends       &         1 \\
Preços              &         1 \\
Expectativas        &         1 \\
\bottomrule
\end{tabular}



C:\Users\Henri\AppData\Local\Temp\ipykernel_4368\1246526744.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(robust_data['category'].value_counts().to_latex())


# Verificando as variáveis individuais

In [12]:
# Criando df para mapear grupo no dataframe de contribuição dos componentes

map_vars = meta_data[['name', 'name_df']].copy()
map_vars.set_index('name_df', inplace = True)
map_vars = map_vars.iloc[1:]

robust_vars = pd.merge(series_df, map_vars, left_index=True, right_index=True)
robust_vars

,0,name
vehi_total_sales_fenabrave,0,Vendas de veículos pelas concessionárias - Tot...
imp_comgood_fob,1,Importações - FOB - Bens de consumo
energy_total,2,Consumo de energia elétrica - Brasil - Total\t
road_flow_gross_vehi,3,Tráfego de veículos pesados nas estradas pedag...
gt_viagem,4,Google Trends - Viagem
total_sales,5,Índice volume de vendas no varejo - Total - Br...
ibc_br,6,Índice de Atividade Econômica do Banco Central...
caged_ind_ext,7,Novo Caged - Indústrias extrativas
ind_real_sales,8,Vendas industriais reais (1992=100) - Indústri...
ind_prod,10,Indicadores da produção (2012=100) - Produção ...
